In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

22/02/01 13:03:08 WARN Utils: Your hostname, oussama-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface wlp3s0)
22/02/01 13:03:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/01 13:03:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("book_id", IntegerType(), True),
    StructField("is_read", IntegerType(), True),
    StructField("rating", FloatType(), True)])


ratings = spark.read.csv("data/books_ratings.csv", header=True, schema=schema)
(training, test) = ratings.randomSplit([0.8, 0.2])

In [4]:
ratings.show()

22/02/01 13:03:33 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/oussama/Desktop/workspace/matrix_factorization_recommender/data/books_ratings.csv


+-------+-------+-------+------+
|user_id|book_id|is_read|rating|
+-------+-------+-------+------+
|      0|    948|      1|   5.0|
|      0|    947|      1|   5.0|
|      0|    946|      1|   5.0|
|      0|    945|      1|   5.0|
|      0|    944|      1|   5.0|
|      0|    943|      1|   5.0|
|      0|    942|      1|   5.0|
|      0|    941|      1|   5.0|
|      0|    940|      1|   5.0|
|      0|    939|      1|   5.0|
|      0|    938|      1|   5.0|
|      0|    937|      1|   4.0|
|      0|    936|      1|   4.0|
|      0|    935|      1|   4.0|
|      0|    934|      1|   5.0|
|      0|    933|      1|   4.0|
|      0|    932|      1|   4.0|
|      0|    931|      1|   5.0|
|      0|    930|      1|   2.0|
|      0|    929|      1|   4.0|
+-------+-------+-------+------+
only showing top 20 rows



In [5]:
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="book_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

22/02/01 13:03:50 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/oussama/Desktop/workspace/matrix_factorization_recommender/data/books_ratings.csv
22/02/01 13:03:56 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/oussama/Desktop/workspace/matrix_factorization_recommender/data/books_ratings.csv


In [6]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

22/02/01 13:04:33 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/oussama/Desktop/workspace/matrix_factorization_recommender/data/books_ratings.csv


Root-mean-square error = 2.225046600971715


In [7]:
movieRecs = model.recommendForAllItems(1)

/home/oussama/.local/share/virtualenvs/matrix_factorization_recommender-qyAKTrSQ/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [8]:
movieRecs.show()

+-------+-------------------+
|book_id|    recommendations|
+-------+-------------------+
|      1|         [{0, 0.0}]|
|     12|[{1775, 5.1165895}]|
|     13|  [{58, 10.170677}]|
|     22| [{545, 3.8608067}]|
|     26| [{329, 5.6786933}]|
|     27|         [{0, 0.0}]|
|     28|  [{58, 12.186274}]|
|     31|  [{58, 7.4835396}]|
|     34|         [{0, 0.0}]|
|     44|         [{0, 0.0}]|
|     47|  [{181, 8.949086}]|
|     52|[{1249, 4.8477545}]|
|     53|[{1760, 14.496603}]|
|     65|         [{0, 0.0}]|
|     76|         [{0, 0.0}]|
|     78|   [{509, 9.35182}]|
|     81|         [{0, 0.0}]|
|     85| [{545, 6.4346776}]|
|     91|         [{0, 0.0}]|
|     93|[{1111, 10.883413}]|
+-------+-------------------+
only showing top 20 rows

